# Bibliotecas

In [ ]:
from obterDados import obterSimboloPosicao, obterSimboloData
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

# Obtendo dados

In [ ]:
rawDataM5 = obterSimboloPosicao()
rawDataM5

In [ ]:
# rawDataM5[:-50*108]

In [ ]:
rawDataM5['minute'] = rawDataM5.index.minute
rawDataM5['hour'] = rawDataM5.index.hour
rawDataM5['day_of_week'] = rawDataM5.index.day_of_week
rawDataM5['day'] = rawDataM5.index.day
# rawDataM5NP = rawDataM5.to_numpy()
rawDataM5NP = rawDataM5[:-50*108].to_numpy()
rawDataM5NP

In [ ]:
# np.save(open('data/raw/rawDataM5NP.npy', 'wb'), rawDataM5NP)

# Tratamento de Dados

### Gerando colunas com preços passados

In [ ]:
nPassado = 300
nFuturo = 24
histM5NP = np.zeros((len(rawDataM5NP)-(nPassado+nFuturo), nPassado+1, 6))
for i in tqdm(range(len(histM5NP))):
    histM5NP[i, 0, -4:] = rawDataM5NP[i:i+nPassado][-1][-4:]
    histM5NP[i, 1:] = rawDataM5NP[i:i+nPassado, :6]

### Achar quando comprar, vender ou fazer nada

In [ ]:
pontos = 30
y = np.zeros((len(histM5NP)))
for i in tqdm(range(len(y))):
    ultimoPreco = histM5NP[i, -1, 3]
    for j in range(nFuturo):
        # nao fazer nada se for mais de 14hs
        # if rawNP[nPassado+i+j, -3] > 14:
        #     break
        # comprar
        if rawDataM5NP[nPassado+i+j, 1] >= ultimoPreco + pontos:
            y[i] = 1
            break
        # vender
        if rawDataM5NP[nPassado+i+j, 2] <= ultimoPreco - pontos:
            y[i] = 2
            break

# Obtendo histórico D1

In [ ]:
histNP = np.zeros((histM5NP.shape[0], histM5NP.shape[1] + 200, histM5NP.shape[2]))

for i in tqdm(range(len(histNP))):
    rawDataD1 = obterSimboloData(rawDataM5.index[i+nFuturo])
    rawDataD1NP = rawDataD1.to_numpy()
    histNP[i, :histM5NP.shape[1]] = histM5NP[i]
    histNP[i, histM5NP.shape[1]:] = rawDataD1NP
    

### Normalização

In [ ]:
histNPNorm = np.zeros(histNP.shape, dtype=np.float64)
for i in tqdm(range(len(histNPNorm))):
    # preços
    vmax = histNP[i, 1:nPassado+1, :4].max()
    vmin = histNP[i, 1:nPassado+1, :4].min()
    histNPNorm[i, 1:nPassado+1, :4] = (histNP[i, 1:nPassado+1, :4] - vmin) / (vmax - vmin)
    # tick volume
    vmax = histNP[i, 1:nPassado+1, 4].max()
    vmin = histNP[i, 1:nPassado+1, 4].min()
    histNPNorm[i, 1:nPassado+1, 4] = (histNP[i, 1:nPassado+1, 4] - vmin) / (vmax - vmin)
    # real volume
    vmax = histNP[i, 1:nPassado+1, 5].max()
    vmin = histNP[i, 1:nPassado+1, 5].min()
    histNPNorm[i, 1:nPassado+1, 5] = (histNP[i, 1:nPassado+1, 5] - vmin) / (vmax - vmin)
    # preços
    vmax = histNP[i, nPassado+1:, :4].max()
    vmin = histNP[i, nPassado+1:, :4].min()
    histNPNorm[i, nPassado+1:, :4] = (histNP[i, nPassado+1:, :4] - vmin) / (vmax - vmin)
    # tick volume
    vmax = histNP[i, nPassado+1:, 4].max()
    vmin = histNP[i, nPassado+1:, 4].min()
    histNPNorm[i, nPassado+1:, 4] = (histNP[i, nPassado+1:, 4] - vmin) / (vmax - vmin)
    # real volume
    vmax = histNP[i, nPassado+1:, 5].max()
    vmin = histNP[i, nPassado+1:, 5].min()
    histNPNorm[i, nPassado+1:, 5] = (histNP[i, nPassado+1:, 5] - vmin) / (vmax - vmin)
# minuto
histNPNorm[:, 0, 2] = histNP[:, 0, 2] / 60
# hora
histNPNorm[:, 0, 3] = histNP[:, 0, 3] / 24
# dia da semana
histNPNorm[:, 0, 4] = histNP[:, 0, 4] / 4
# dia
histNPNorm[:, 0, 5] = histNP[:, 0, 5] / 31


### Sem Reshape para 2d no caso para CNNs

In [ ]:
x = histNPNorm

### Balanceamento de Classes

In [ ]:
unicos, contagem = np.unique(y, return_counts=True)
print(contagem)
plt.pie(contagem, labels=unicos)

In [ ]:
amostra = contagem.min()
Amostragem = np.zeros((amostra * 3, x.shape[1], x.shape[2] + 1), dtype=np.float64)

for i in range(3):
    xClasse_i = x[y == i]
    xAmostra = xClasse_i[np.random.choice(xClasse_i.shape[0], size=amostra, replace=False)]
    Amostragem[i*amostra:(i+1)*amostra, :, :-1] = xAmostra
    Amostragem[i*amostra:(i+1)*amostra, :, -1] = i

Amostragem.shape

In [ ]:
np.random.shuffle(Amostragem)
x = Amostragem[:, :, :-1]
y = Amostragem[:, :, -1][:, 0]
y

In [ ]:
unicos, contagem = np.unique(y, return_counts=True)
print(contagem)
plt.pie(contagem, labels=unicos)

### Salvando dados em 2D

In [ ]:
np.save(open('data/cnn/x.npy', 'wb'), x)
np.save(open('data/cnn/y.npy', 'wb'), y)

### Salvando dados em 1D

In [ ]:
np.save(open('data/x.npy', 'wb'), np.reshape(x, (x.shape[0], x.shape[1] * x.shape[2])))
np.save(open('data/y.npy', 'wb'), y)

# Salvando a data do treino

In [ ]:
from datetime import datetime

file = open('lastDataTrained.txt', 'w')
file.write(str(datetime.now().strftime('%Y-%m-%d')))
file.close()